


# Assignment 5 on Natural Language Processing

## Date : 3rd Nov, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

The central idea of this assignment is to explore various language models specifically LSTM based and transformer. We will explore how the size of the model effects the sequence generated. We will see both character based and word based models.



Please submit with outputs. Submissions without predicted outputs will be penalized.

# Word Based LSTM model

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import numpy
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer

Do basic pre processing which includes lowering etc
Check the dataset and apply suitable preprocessing.

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Load the data and preprocess data and store corpus in raw_text
file= open("/content/corpus.txt","r")
raw_text=file.read()
file.close()
raw_text=raw_text.lower()
raw_text=raw_text.replace('_','')
print(raw_text)

chapter i.
down the rabbit-hole alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, “and what is the use of a book,” thought alice “without pictures or conversations?” so she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a white rabbit with pink eyes ran close by her.
there was nothing so very remarkable in that; nor did alice think it so very much out of the way to hear the rabbit say to itself, “oh dear!
oh dear!
i shall be late!” (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the rabbit actually took a watch out of its waistcoat-

In [4]:
temp_tokenizer = RegexpTokenizer(r'\w+')
words=temp_tokenizer.tokenize(raw_text)
unique=set(words)
print("Number of unique words after preprocessing: ",len(unique))

Number of unique words after preprocessing:  2568


In [5]:
# Hyperparameters of the model
vocab_size = 2570 # choose based on statistics
oov_tok = '<OOV>'
embedding_dim = 100
padding_type='post'
trunc_type='post'

In [6]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts([raw_text])
word_index = tokenizer.word_index

In [7]:
seq_length = 50
tokens = tokenizer.texts_to_sequences([raw_text])[0]

In [8]:
print(word_index)

{'<OOV>': 1, 'the': 2, '”': 3, 'and': 4, 'to': 5, 'a': 6, 'she': 7, 'of': 8, 'it': 9, 'said': 10, 'alice': 11, 'in': 12, 'was': 13, 'you': 14, 'i': 15, 'that': 16, 'as': 17, 'her': 18, 'at': 19, 'on': 20, 'with': 21, 'had': 22, 'all': 23, 'be': 24, 'for': 25, 'so': 26, 'very': 27, 'not': 28, 'this': 29, 'little': 30, 'but': 31, '“i': 32, 'out': 33, 'they': 34, 'he': 35, 'down': 36, 'what': 37, 'up': 38, 'is': 39, 'one': 40, 'his': 41, 'about': 42, 'were': 43, 'like': 44, 'went': 45, 'herself': 46, 'them': 47, 'again': 48, 'know': 49, 'then': 50, 'would': 51, 'could': 52, 'have': 53, 'no': 54, 'thought': 55, 'when': 56, 'if': 57, 'do': 58, 'time': 59, 'or': 60, 'there': 61, 'queen': 62, 'into': 63, 'me': 64, 'see': 65, 'off': 66, 'king': 67, 'your': 68, 'did': 69, 'began': 70, 'its': 71, 'by': 72, 'an': 73, 'my': 74, 'mock': 75, '“and': 76, 'quite': 77, 'hatter': 78, 'turtle': 79, 'gryphon': 80, 'way': 81, 'who': 82, 'don’t': 83, 'their': 84, 'think': 85, 'much': 86, 'say': 87, 'some': 

In [9]:
dataX = []
dataY = []

for i in range(0, len(tokens) - seq_length-1 , 1):
  seq_in = tokens[i:i + seq_length]
  seq_out = tokens[i + seq_length]

  if seq_out==1: #Skip samples where target word is OOV
    continue
    
  dataX.append(seq_in)
  dataY.append(seq_out)
 
N = len(dataX)
print ("Total training data size: ", N)

Total training data size:  27081


In [10]:
X = numpy.array(dataX)

# one hot encode the output variable
y = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [11]:
# with embedding
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=seq_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(vocab_size, activation='softmax')
])
# compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           257000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dense (Dense)                (None, 2570)              331530    
Total params: 673,010
Trainable params: 673,010
Non-trainable params: 0
_________________________________________________________________


In [12]:
from keras import callbacks 
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",  
                                        mode ="min", patience = 5,  
                                        restore_best_weights = True) 

In [13]:
# Use validation split of 0.2 while training
model.fit(X, y, epochs=25, batch_size=128,  validation_split=0.2,callbacks =[earlystopping]) 

Epoch 1/25
170/170 [==============================] - 3s 20ms/step - loss: 6.4390 - accuracy: 0.0497 - val_loss: 6.0122 - val_accuracy: 0.0833
Epoch 2/25
170/170 [==============================] - 3s 15ms/step - loss: 5.9814 - accuracy: 0.0535 - val_loss: 5.9149 - val_accuracy: 0.0840
Epoch 3/25
170/170 [==============================] - 3s 15ms/step - loss: 5.8647 - accuracy: 0.0563 - val_loss: 5.8705 - val_accuracy: 0.0951
Epoch 4/25
170/170 [==============================] - 3s 15ms/step - loss: 5.7375 - accuracy: 0.0704 - val_loss: 5.7840 - val_accuracy: 0.1078
Epoch 5/25
170/170 [==============================] - 3s 15ms/step - loss: 5.5834 - accuracy: 0.0933 - val_loss: 5.7035 - val_accuracy: 0.1198
Epoch 6/25
170/170 [==============================] - 3s 15ms/step - loss: 5.4428 - accuracy: 0.1041 - val_loss: 5.6660 - val_accuracy: 0.1241
Epoch 7/25
170/170 [==============================] - 3s 15ms/step - loss: 5.3105 - accuracy: 0.1148 - val_loss: 5.6140 - val_accuracy: 0.1325

In [14]:
word_map=tokenizer.word_index
reverse_word_map = {v:k for k,v in word_map.items()} ## Create word to idx map using tokenizer.word_index

In [15]:
print(reverse_word_map)

{1: '<OOV>', 2: 'the', 3: '”', 4: 'and', 5: 'to', 6: 'a', 7: 'she', 8: 'of', 9: 'it', 10: 'said', 11: 'alice', 12: 'in', 13: 'was', 14: 'you', 15: 'i', 16: 'that', 17: 'as', 18: 'her', 19: 'at', 20: 'on', 21: 'with', 22: 'had', 23: 'all', 24: 'be', 25: 'for', 26: 'so', 27: 'very', 28: 'not', 29: 'this', 30: 'little', 31: 'but', 32: '“i', 33: 'out', 34: 'they', 35: 'he', 36: 'down', 37: 'what', 38: 'up', 39: 'is', 40: 'one', 41: 'his', 42: 'about', 43: 'were', 44: 'like', 45: 'went', 46: 'herself', 47: 'them', 48: 'again', 49: 'know', 50: 'then', 51: 'would', 52: 'could', 53: 'have', 54: 'no', 55: 'thought', 56: 'when', 57: 'if', 58: 'do', 59: 'time', 60: 'or', 61: 'there', 62: 'queen', 63: 'into', 64: 'me', 65: 'see', 66: 'off', 67: 'king', 68: 'your', 69: 'did', 70: 'began', 71: 'its', 72: 'by', 73: 'an', 74: 'my', 75: 'mock', 76: '“and', 77: 'quite', 78: 'hatter', 79: 'turtle', 80: 'gryphon', 81: 'way', 82: 'who', 83: 'don’t', 84: 'their', 85: 'think', 86: 'much', 87: 'say', 88: 'som

In [16]:
print(X.shape)

(27081, 50)


In [17]:
print(y.shape)

(27081, 2570)


In [18]:
# Complete the code to return next n words greedily
def next_tokens(pattern, input_str, n): 
	print ("Seed: \n",  input_str)
	print("Prediction: ")
	final_string=''
	for i in range(n):
		x=numpy.array(pattern)
		prediction = model.predict( x , verbose=0)
		b= np.argmax(model.predict(x), axis=-1)
		# get next word index. Use reverse_word_map to get the word
		word= reverse_word_map[b[len(b)-1]]
		final_string+=word
		final_string+=' '
		pattern.append(b[len(b)-1])
		pattern = pattern[1:len(pattern)]
	return final_string

In [19]:
# pick a random seed

start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ' '.join([reverse_word_map[value] for value in pattern])
print(next_tokens(pattern,input_str , 10))

Seed: 
 he had taken his watch out of his pocket and was looking at it uneasily shaking it every now and then and holding it to his ear alice considered a little and then said “the fourth ” “two days wrong ” sighed the hatter “i told you butter wouldn’t suit
Prediction: 
the little little little little little little little little little 


In [20]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

# Use first 50 tokens from given input_str as input.(Use tokenizer to split to take first 50)
input_str=input_str.lower()
word_tokens = tokenizer.texts_to_sequences([input_str])[0]
seq= word_tokens[:50]
inp=' '.join([reverse_word_map[val] for val in seq])
print(next_tokens( seq, inp,10))

Seed: 
 the boy laughed at the fright he had caused this time the <OOV> left angrily the <OOV> day as the boy went up the small <OOV> he suddenly saw a <OOV> <OOV> his <OOV> he cried as hard as he could <OOV> <OOV> <OOV> ” but not a <OOV> <OOV>
Prediction: 
thing to the little little little little little little little 


# Character based LSTM Model 1

In [21]:
# User the preprocess data and create raw_text

# create mapping of unique characters to integers
chars = sorted(list(set(raw_text)))

char_to_int = dict((ch, i) for i, ch in enumerate(chars))

In [22]:
# Print the total characters and character vacob size
n_chars = len(raw_text)
n_vocab = len(chars)

In [23]:
'''
Prepare dataset where the input is sequence of 100 characters and target is next character.
'''
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    # Write code here
    seq_in= raw_text[i:i+seq_length]
    seq_out= raw_text[i+seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  141937


In [24]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length,1))

# one hot encode the output variable
dataY = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [25]:
embedding_dim =100
max_length =100

In [26]:
model = Sequential()
model.add(keras.layers.Embedding(n_vocab, embedding_dim, input_length=max_length))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          4500      
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 45)                11565     
Total params: 381,633
Trainable params: 381,633
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.fit(X, y, epochs=20, batch_size=128)

Epoch 1/20
1109/1109 [==============================] - 20s 18ms/step - loss: 2.1937
Epoch 2/20
1109/1109 [==============================] - 20s 18ms/step - loss: 1.6957
Epoch 3/20
1109/1109 [==============================] - 20s 18ms/step - loss: 1.4956
Epoch 4/20
1109/1109 [==============================] - 20s 18ms/step - loss: 1.3790
Epoch 5/20
1109/1109 [==============================] - 20s 18ms/step - loss: 1.2965
Epoch 6/20
1109/1109 [==============================] - 20s 18ms/step - loss: 1.2375
Epoch 7/20
1109/1109 [==============================] - 20s 18ms/step - loss: 1.1869
Epoch 8/20
1109/1109 [==============================] - 20s 18ms/step - loss: 1.1416
Epoch 9/20
1109/1109 [==============================] - 20s 18ms/step - loss: 1.1028
Epoch 10/20
1109/1109 [==============================] - 20s 18ms/step - loss: 1.0690
Epoch 11/20
1109/1109 [==============================] - 20s 18ms/step - loss: 1.0378
Epoch 12/20
1109/1109 [==============================] - 20s 18

In [28]:
#implement mapping of integer to character
int_to_char = dict((i,ch) for i,ch in enumerate(chars))

In [29]:
'''
Complete code below to get the generated string using the model.
'''
def predict_next_100_chars(pattern,x):
	print("Prediction: ")
	final_string=''
	for i in range(x):
		# Complete Code
		a=numpy.reshape(pattern,(1, len(pattern),1))
		prediction = model.predict(a, verbose=0)
		idx=numpy.argmax(prediction)
		ch= int_to_char[idx]
		seq_in=[int_to_char[b] for b in pattern]
		final_string+=ch
		pattern.append(idx)
		pattern=pattern[1: len(pattern)]
	return final_string

In [30]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])

print(predict_next_100_chars(pattern,200))

Prediction: 
, they lived out to the same thing as ‘i eat or down, and the moral of that is—‘the dormouse said—” she said to herself, “that is it i can have trial, you know what the dormouse said—” “then you can h


In [31]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

 # Use first 100 characeters from given input_str as input and generate next 200 characters.
input_str=input_str.lower()
string= input_str[:100]
char_seq=[char_to_int[c] for c in string]
print(predict_next_100_chars(char_seq,200))

Prediction: 
s it was a little shriek, and was going to get to the table in the words:— “i don’t think it would be a little good many muchess to be a little passage, and the same thing i am on the song, were all t


## Character based LSTM Model 2


In [32]:
model1 = Sequential()
model1.add(keras.layers.Embedding(n_vocab, embedding_dim, input_length=max_length))
model1.add(LSTM(256, input_shape=(X.shape[1], embedding_dim),return_sequences=True))
model1.add(Dropout(0.2))
model1.add(LSTM(256))
model1.add(Dropout(0.2))
model1.add(Dense(y.shape[1], activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam')

In [33]:
model1.fit(X, y, epochs=20, batch_size=64)

Epoch 1/20
2218/2218 [==============================] - 50s 23ms/step - loss: 2.1183
Epoch 2/20
2218/2218 [==============================] - 50s 22ms/step - loss: 1.5611
Epoch 3/20
2218/2218 [==============================] - 50s 23ms/step - loss: 1.3791
Epoch 4/20
2218/2218 [==============================] - 50s 22ms/step - loss: 1.2768
Epoch 5/20
2218/2218 [==============================] - 50s 23ms/step - loss: 1.2089
Epoch 6/20
2218/2218 [==============================] - 50s 23ms/step - loss: 1.1514
Epoch 7/20
2218/2218 [==============================] - 50s 23ms/step - loss: 1.1110
Epoch 8/20
2218/2218 [==============================] - 50s 22ms/step - loss: 1.0727
Epoch 9/20
2218/2218 [==============================] - 50s 22ms/step - loss: 1.0386
Epoch 10/20
2218/2218 [==============================] - 50s 23ms/step - loss: 1.0086
Epoch 11/20
2218/2218 [==============================] - 50s 22ms/step - loss: 0.9859
Epoch 12/20
2218/2218 [==============================] - 50s 23

In [45]:
# Generate the sequence similar to above methods

'''
Complete code below to get the generated string using the model.
'''
def predict_next_100_Chars(pattern,x):
	print("Prediction: ")
	final_string=''
	for i in range(x):
		# Complete Code
		a=numpy.reshape(pattern,(1, len(pattern),1))
		prediction = model1.predict(a, verbose=0)
		idx=numpy.argmax(prediction)
		ch= int_to_char[idx]
		seq_in=[int_to_char[b] for b in pattern]
		final_string+=ch
		pattern.append(idx)
		pattern=pattern[1: len(pattern)]

	return final_string

In [46]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])

print(predict_next_100_Chars(pattern,200))

Prediction: 
 of the garden, and she was now about it a book of the song.
“what a can remark many more than that is, if i only wish the moral of that dormouse is it,” said the mouse, who was not gone with the door


In [47]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

 # Use first 100 characeters from given input_str as input and generate next 200 characters.
input_str=input_str.lower()
string= input_str[:100] 
char_seq=[char_to_int[c] for c in string]
 
print(predict_next_100_Chars(char_seq,200))

Prediction: 
s she was now about it a book of the song.
“what a can remark many more than that is, if i only wish the moral of that dormouse is it,” said the mouse, who was not gone with the door and shouted to th


# Questions

**Question:** What are your observations based on the model(all) outputs on train data(in domain) vs unseen data(out of domain) ?

**Answer:** The model performs much better on train data which it has seen than on unseen data since it labels unseen tokens on 'OOV' and hence fails to generalize well. 

**Question:** What did you observe in the outputs of char LSTM model1 vs char LSTM model2 ?

**Answer:** LSTM model2 generalizes well and is able to model the character dependencies much better than LSTM model1

# Bonus (Not to be graded)

## Transformer based language model (Bert)
You can run the below cells to predict the next word using pretrained BERT model. 

In [37]:
!pip install transformers

In [38]:
import os
import torch
import string
from transformers import BertTokenizer, BertForMaskedLM

In [39]:
def load_model(model_name):
  try:
    if model_name.lower() == "bert":
      bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
      bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()
      return bert_tokenizer,bert_model
  except Exception as e:
    pass

In [40]:
def decode(tokenizer, pred_idx, top_clean):
  ignore_tokens = string.punctuation + '[PAD]'
  tokens = []
  for w in pred_idx:
    token = ''.join(tokenizer.decode(w).split())
    if token not in ignore_tokens:
      tokens.append(token.replace('##', ''))
  return '\n'.join(tokens[:top_clean])

In [41]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
  text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
  # if <mask> is the last token, append a "." so that models dont predict punctuation.
  if tokenizer.mask_token == text_sentence.split()[-1]:
    text_sentence += ' .'
  input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
  mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
  return input_ids, mask_idx

In [42]:
def get_all_predictions(text_sentence, top_clean=5):
  input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
  with torch.no_grad():
    predict = bert_model(input_ids)[0]
    print(predict.shape)
    
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)
  return {'bert': bert}

In [43]:
def get_prediction_eos(input_text):
  try:
    input_text += ' <mask>'
    res = get_all_predictions(input_text, top_clean=int(top_k))
    return res
  except Exception as error:
    pass

In [44]:
# Below code predicts the next top_k words. You can modify this to get next n words using top_k=1 and greedy decoding it. 
top_k= 3
print('Predict next ', top_k, ' words')
model_name = 'BERT'
bert_tokenizer, bert_model  = load_model(model_name) 
input_text = "" ### GIVE YOUR INPUT STRING HERE
res = get_prediction_eos(input_text)
answer = []
print(res['bert'].split("\n"))
for i in res['bert'].split("\n"):
  answer.append(i)
  answer_as_string = "    ".join(answer)

print(answer_as_string)

Predict next  3  words


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([1, 4, 30522])
['p', 'vol', 'math']
p    vol    math
